In [1]:
!pwd

/home/iceberg/notebooks


In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("spark-iceberg-minio-dev")

    # Iceberg catalog (HadoopCatalog on MinIO)
    .config("spark.sql.catalog.local_minio", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.local_minio.catalog-impl", "org.apache.iceberg.hadoop.HadoopCatalog")
    .config("spark.sql.catalog.local_minio.warehouse", "s3a://warehouse/")

    # Iceberg SQL extensions (mandatory)
    .config(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
    )

    .getOrCreate()
)

[02:10:39] [Thread-3] WARN  org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
hadoop_conf = spark._jsc.hadoopConfiguration()

hadoop_conf.set("fs.s3a.endpoint", "http://minio:9000")
hadoop_conf.set("fs.s3a.access.key", "minioadmin")
hadoop_conf.set("fs.s3a.secret.key", "minioadmin")
hadoop_conf.set("fs.s3a.path.style.access", "true")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

# Good defaults
hadoop_conf.set("fs.s3a.connection.ssl.enabled", "false")
hadoop_conf.set("fs.s3a.aws.credentials.provider",
                 "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

In [4]:
spark.sparkContext.setLogLevel("WARN")

In [5]:
spark.version

'4.0.1'

In [6]:
spark.sql("SHOW CATALOGS").show()

+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [7]:
spark.read.text("s3a://warehouse/").show(5)

[02:11:28] [Thread-3] WARN  org.apache.hadoop.metrics2.impl.MetricsConfig - Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
+-----+
|value|
+-----+
+-----+



In [8]:
df = spark.createDataFrame(
    [(1, "a"), (2, "b"), (3, "c")],
    ["id", "value"]
)

df.writeTo("local_minio.test_db.test_table").create()

[02:11:35] [Thread-3] INFO  org.apache.iceberg.CatalogUtil - Loading custom FileIO implementation: org.apache.iceberg.hadoop.HadoopFileIO
[02:11:35] [Thread-3] INFO  org.apache.iceberg.BaseMetastoreCatalog - Table properties set at catalog level through catalog properties: {}
[02:11:35] [Thread-3] INFO  org.apache.iceberg.BaseMetastoreCatalog - Table properties enforced at catalog level through catalog properties: {}


[02:11:40] [Thread-3] INFO  org.apache.iceberg.spark.source.SparkWrite - Committing append with 3 new data files to table test_db.test_table
[02:11:40] [Thread-3] INFO  org.apache.iceberg.SnapshotProducer - Committed snapshot 2658278754181055859 (MergeAppend)
[02:11:40] [Thread-3] INFO  org.apache.iceberg.metrics.LoggingMetricsReporter - Received metrics report: CommitReport{tableName=test_db.test_table, snapshotId=2658278754181055859, sequenceNumber=1, operation=append, commitMetrics=CommitMetricsResult{totalDuration=TimerResult{timeUnit=NANOSECONDS, totalDuration=PT0.337192287S, count=1}, attempts=CounterResult{unit=COUNT, value=1}, addedDataFiles=CounterResult{unit=COUNT, value=3}, removedDataFiles=null, totalDataFiles=CounterResult{unit=COUNT, value=3}, addedDeleteFiles=null, addedEqualityDeleteFiles=null, addedPositionalDeleteFiles=null, addedDVs=null, removedDeleteFiles=null, removedEqualityDeleteFiles=null, removedPositionalDeleteFiles=null, removedDVs=null, totalDeleteFiles=Cou

In [9]:
spark.read.table("local_minio.test_db.test_table").show()

[02:11:47] [Thread-3] INFO  org.apache.iceberg.BaseMetastoreCatalog - Table loaded by catalog: local_minio.test_db.test_table
[02:11:47] [Thread-3] INFO  org.apache.iceberg.SnapshotScan - Scanning table local_minio.test_db.test_table snapshot 2658278754181055859 created at 2025-12-30T02:11:40.245+00:00 with filter true
[02:11:47] [Thread-3] INFO  org.apache.iceberg.BaseDistributedDataScan - Planning file tasks locally for table local_minio.test_db.test_table
[02:11:47] [Thread-3] INFO  org.apache.iceberg.spark.source.SparkPartitioningAwareScan - Reporting UnknownPartitioning with 1 partition(s) for table local_minio.test_db.test_table
[02:11:47] [Executor task launch worker for task 0.0 in stage 1.0 (TID 12)] INFO  org.apache.iceberg.spark.data.vectorized.VectorizedSparkParquetReaders - Enabling arrow.enable_unsafe_memory_access
[02:11:47] [Executor task launch worker for task 0.0 in stage 1.0 (TID 12)] INFO  org.apache.iceberg.spark.data.vectorized.VectorizedSparkParquetReaders - Disa